In [1]:
import pandas as pd

In [15]:
df = pd.read_csv('/Users/drewnleonard/Documents/thesis/data/csv/fb_gold.csv')

In [16]:
##      source target weight
## 1     C-3PO  R2-D2     17
## 2      LUKE  R2-D2     13
## 3   OBI-WAN  R2-D2      6
## 4      LEIA  R2-D2      5
## 5       HAN  R2-D2      5
## 6 CHEWBACCA  R2-D2      3


In [17]:
# Remove unavailable
df = df[df["AccountGroup"] != 'Unavailable']
df = df[df["Interests"] != 'unavailable']
df = df[['AccountGroup','Interests']]

In [20]:
group_to_interest = {}
for n, e in df.iterrows():
    
    group = e['AccountGroup']
    interest = e['Interests']
    
    if group not in group_to_interest:
        group_to_interest[group] = {}
    
    if interest not in group_to_interest[group]:
        group_to_interest[group][interest] = 0
    
    group_to_interest[group][interest] += 1

In [25]:
group_to_group = {}
for group,interest_map in group_to_interest.iteritems():
    for interest,count in interest_map.iteritems():
        
        for inner_group,inner_interest_map in group_to_interest.iteritems():
            for inner_interest,inner_count in inner_interest_map.iteritems():
                
                if group != inner_group and interest == inner_interest:
                    shared = min(count,inner_count)
                    
                    if inner_group in group_to_group:
                        if group not in group_to_group[inner_group]:
                            group_to_group[inner_group][group] = 0
                        group_to_group[inner_group][group] += shared
                    elif group in group_to_group:
                        if inner_group not in group_to_group[group]:
                            group_to_group[group][inner_group] = 0
                        group_to_group[group][inner_group] += shared
                    else:
                        group_to_group[group] = {}
                        group_to_group[group][inner_group] = shared
                    

In [35]:
group1l = []
group2l = []
weightl = []

for group1, group1_map in group_to_group.iteritems():
    for group2, weight in group1_map.iteritems():
        
        group1l.append(group1)
        group2l.append(group2)
        weightl.append(weight)

In [36]:
group_to_group_df = pd.DataFrame()
group_to_group_df['group1'] = group1l
group_to_group_df['group2'] = group2l
group_to_group_df['weight'] = weightl

In [50]:
node_name = []
node_id = []
n = 0
seen = set()
for n, e in df.iterrows():
    group = e['AccountGroup']
    if group not in seen:
        node_name.append(group)
        node_id.append(n)
        n += 1
    seen.add(group)

In [51]:
node_df = pd.DataFrame()
node_df['node'] = node_name
node_df['id'] = node_id

In [53]:
node_df.to_csv('/Users/drewnleonard/Documents/thesis/data/csv/group_node.csv',index=False)
group_to_group_df.to_csv('/Users/drewnleonard/Documents/thesis/data/csv/group_to_group.csv',index=False)

In [54]:
node_df.shape

(61, 2)

In [56]:
group_to_group_df.shape

(622, 3)